In [1]:
#code to get names of the images in dataset so that lables can be assigned
import pandas as pd
import os

#defining the address of the dataset
DATA_DIR = "E:\\train"
names=[]
i=0
for root, dirs, files in os.walk(DATA_DIR,topdown=False):
    for name in files:
        names.append(name.replace('.jpeg',''))
        #print(names[i])
        #i+=1
print(len(names))
df_Org=pd.read_csv("E:\\trainLabels.csv\\trainLabels.csv")
sf=df_Org.values.tolist()
#print(sf)

1427


In [2]:
#extracting the labels for all the images in the dataset
df=[]
for name in names:
    for i in range (0,len(sf)):
        if(sf[i][0]==name):
            df.append(sf[i])
print(len(df))

1427


In [3]:
#getting all the images and creating the dataset arrays x and y
import cv2
import numpy as np
x=[]
y=[]
for i in range (0,len(df)-1):
    #raeding in the image
    img = cv2.imread("E://train//"+df[i][0]+".jpeg")
    #resizing the image
    img = cv2.resize(img,(512,340))
    #changing the array to a numpy n-dimensional array and setting its shape
    img = np.array(img).reshape((340,512,3))
    #adding the image to the training data array
    x.append(img)
    #adding the labels
    y.append(df[i][1])
    i+=1
x=np.array(x)
y=np.array(y)
y=np.array(y).reshape(len(y),1)
print(x.shape)
print(y.shape)

(1426, 340, 512, 3)
(1426, 1)


In [4]:
#splitting up the data into training and testing data
from sklearn.model_selection import train_test_split as tts
x_train, x_test, y_train, y_test = tts(x, y, test_size=0.2987, random_state=42)
X_train=np.split(x_train,10)
y_train=np.split(y_train,10)
print(np.array(X_train).shape)
print(np.array(y_train).shape)

(10, 100, 340, 512, 3)
(10, 100, 1)


In [5]:
#normalizaruion function for the dataset
def normalize(x):
    max_x,min_x=x.max(),x.min()
    norm_x=np.zeros(tuple(x.shape))
    norm_img=x.shape[0]
    for nr in range(norm_img):
        norm_x[nr,...]=(x[nr,...]-float(min_x))/(float(max_x-min_x))
    return norm_x

#function for one hot encoding the labels
def one_hot_encode(x):
    from sklearn.preprocessing import OneHotEncoder
    enc=OneHotEncoder(n_values=5)
    ohc_labels=enc.fit_transform(np.array(x).reshape(-1,1)).toarray()
    return ohc_labels

In [6]:
import tensorflow as tf

#declaring tensorflow global variables
x = tf.placeholder("float", shape=[None,340,512,3], name='x')
y = tf.placeholder("float", shape=[None,5], name='y')
keep_probs = tf.placeholder("float", name='keep_probs')

#defining the convolution 2d layer
def conv2d(inp_lyr,ofilters,kernel,stride):
    conv2d_lyr=tf.layers.conv2d(inputs=inp_lyr,
                               filters=ofilters,
                               kernel_size=kernel,
                               strides=stride,
                               padding="same",
                               activation=tf.nn.relu,
                               )
    return conv2d_lyr

#defing the maxpool 2d layer
def maxpool2d(inp_lyr,pool,stride):
    maxp2d=tf.layers.max_pooling2d(inputs=inp_lyr,
                                 pool_size=pool,
                                 strides=stride,
                                 padding="same")
    return maxp2d

c:\users\hp\anaconda3\envs\testenv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
def model():
    
    #getting the input
    img_inp=tf.reshape(x,[-1,340,512,3])
    #one convolution and one maxpool layer
    layer=conv2d(img_inp,32,[7,7],[2,2])
    layer=maxpool2d(layer,[5,5],[2,2])
    
    #two convolution and one maxpool layer
    layer=conv2d(layer,32,[5,5],[1,1])
    layer=conv2d(layer,64,[3,3],[1,1])
    layer=maxpool2d(layer,[3,3],[2,2])
    
    #three convolution and one maxpool layer
    layer=conv2d(layer,64,[3,3],[1,1])
    layer=conv2d(layer,128,[3,3],[1,1])
    layer=conv2d(layer,256,[3,3],[1,1])
    layer=maxpool2d(layer,[3,3],[2,2])
    
    #dropout layer
    layer=tf.layers.dropout(layer,0.4)
    
    #flattening layer
    layer=tf.layers.flatten(layer)
    
    #dense (fully connected) layers
    layer=tf.layers.dense(layer,256)
    layer=tf.layers.dense(layer,512)
    layer=tf.layers.dense(layer,1024)
    
    layer=tf.layers.dense(layer,5)
    
    #activation layer
    layer=tf.nn.softmax(layer)
    
    return layer

In [8]:
#defining the global training variables
cnn_mod=model()
cnn_mod=tf.identity(cnn_mod, name='model')
cross_entropy=-tf.reduce_sum(y*tf.log(cnn_mod))
epoch_batch=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
corr_pred=tf.equal(tf.argmax(cnn_mod, 1), tf.argmax(y, 1))
acc=tf.reduce_mean(tf.cast(corr_pred, tf.float32), name='acc')

In [9]:
#the function which will be called for training the model
def train_model():
    for epoch in range(15):
        for i in range (0,10):
            x_img=normalize(X_train[i])
            y_img=one_hot_encode(y_train[i])
            tr_acc=sess.run(acc,feed_dict={x: x_img, y: y_img, keep_probs: 1.0})
            print('Epoch: {} Batch: {} Accuracy = {:.6f}'.format(epoch+1, i+1, tr_acc))
            sess.run(epoch_batch,feed_dict={x: x_img, y: y_img, keep_probs: 0.5})

In [10]:
#declaring a session, initializing it and training the model
sess=tf.Session()
sess.run(tf.global_variables_initializer())    
train_model()

Epoch: 1 Batch: 1 Accuracy = 0.060000
Epoch: 1 Batch: 2 Accuracy = 0.740000
Epoch: 1 Batch: 3 Accuracy = 0.700000
Epoch: 1 Batch: 4 Accuracy = 0.760000
Epoch: 1 Batch: 5 Accuracy = 0.720000
Epoch: 1 Batch: 6 Accuracy = 0.690000
Epoch: 1 Batch: 7 Accuracy = 0.670000
Epoch: 1 Batch: 8 Accuracy = 0.770000
Epoch: 1 Batch: 9 Accuracy = 0.710000
Epoch: 1 Batch: 10 Accuracy = 0.770000
Epoch: 2 Batch: 1 Accuracy = 0.710000
Epoch: 2 Batch: 2 Accuracy = 0.740000
Epoch: 2 Batch: 3 Accuracy = 0.700000
Epoch: 2 Batch: 4 Accuracy = 0.760000
Epoch: 2 Batch: 5 Accuracy = 0.720000
Epoch: 2 Batch: 6 Accuracy = 0.690000
Epoch: 2 Batch: 7 Accuracy = 0.670000
Epoch: 2 Batch: 8 Accuracy = 0.770000
Epoch: 2 Batch: 9 Accuracy = 0.710000
Epoch: 2 Batch: 10 Accuracy = 0.770000
Epoch: 3 Batch: 1 Accuracy = 0.710000
Epoch: 3 Batch: 2 Accuracy = 0.740000
Epoch: 3 Batch: 3 Accuracy = 0.700000
Epoch: 3 Batch: 4 Accuracy = 0.760000
Epoch: 3 Batch: 5 Accuracy = 0.720000
Epoch: 3 Batch: 6 Accuracy = 0.690000
Epoch: 3 B

In [11]:
#saving the model so that it can be read back later on
save_model_path='F:/Work/capstone'
saver = tf.train.Saver()
save_path = saver.save(sess, save_model_path)

In [12]:
#preparing the test data
X_test=x_test[:400]
Y_test=y_test[:400]
x_test=np.split(X_test,4)
y_test=np.split(Y_test,4)

In [13]:
#defining the function that will test the model on unknown data
def test_model():
    
    loaded_graph = tf.Graph()

    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)

        # Get Tensors from loaded model
        loaded_x = loaded_graph.get_tensor_by_name('x:0')
        loaded_y = loaded_graph.get_tensor_by_name('y:0')
        loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_probs:0')
        loaded_acc = loaded_graph.get_tensor_by_name('acc:0')
        
        # Get accuracy in batches for memory limitations
        test_batch_acc_total = 0
        test_batch_count = 0
        
        for i in range(0,4):
            x_test_img=normalize(x_test[i])
            y_test_img=one_hot_encode(y_test[i])
            test_batch_acc_total += sess.run(
                loaded_acc,
                feed_dict={loaded_x: x_test_img, loaded_y: y_test_img, loaded_keep_prob: 1.0})
            test_batch_count += 1

        print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))


test_model()

INFO:tensorflow:Restoring parameters from F:/Work/capstone
Testing Accuracy: 0.6799999922513962

